In [4]:

import pygame
import math
import random
import time

pygame.init()

WIDTH = 800
HEIGHT = 600
screen = pygame.display.set_mode((WIDTH,HEIGHT))
MAX_X = WIDTH // 2 - 20  # Отступ 20 пикселей от края
MAX_Y = HEIGHT // 2 - 20
clock = pygame.time.Clock()

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 100, 255)

font = pygame.font.Font(None, 24)

robot_x = 0
robot_y = 0
robot_direction = 0  # градусы, 0 = вправо
# Скорости
linear_speed = 0  # пикселей/секунду
angular_speed = 0  # градусов/секунду
speed_multiplier = 1.0
control_mode = "keyboard"
def draw_robot(screen, x, y, direction, radius):
    """Рисует робота с направлением"""
    # TODO: Преобразуйте математические координаты в экранные
    screen_x = int(x + WIDTH/2)
    screen_y = int(HEIGHT/2 - y)

    pygame.draw.circle(screen, color=BLUE, center=(screen_x, screen_y), radius=radius)

    angle_rad = math.radians(direction)
    
    # Вершина треугольника (впереди)
    tip_x = screen_x + math.cos(angle_rad) * radius * 1  # используйте screen_x, radius, cos(angle_rad)
    tip_y = screen_y - math.sin(angle_rad) * radius * 1  # используйте screen_y, radius, sin(angle_rad)
    
    # Задние углы треугольника (левый и правый)
    # Подсказка: используйте углы (angle_rad + 2.5) и (angle_rad - 2.5)
    left_x = screen_x + math.cos(angle_rad+2.5) * radius * 0.8
    left_y = screen_y - math.sin(angle_rad+2.5) * radius * 0.8
    
    right_x = screen_x + math.cos(angle_rad-2.5) * radius * 0.8
    right_y = screen_y - math.sin(angle_rad-2.5) * radius * 0.8
    
    pygame.draw.polygon(surface=screen, color=WHITE, points=[
        (tip_x, tip_y),
        (left_x, left_y),
        (right_x, right_y)
    ])
    
    pygame.draw.circle(surface=screen, color=BLACK, center=(screen_x, screen_y), radius=radius, width=2)


def draw_trail(screen, trail):
    """Рисует след за роботом"""
    # TODO: Проверьте, что в следе хотя бы 2 точки
    if len(trail) < 2:
        return  # Нечего рисовать
    
    # TODO: Нарисуйте линии между точками
    for i in range(len(trail) - 1):
        # Получаем две соседние точки
        point1 = trail[i]
        point2 = trail[i + 1]
        
        # TODO: Преобразуем математические координаты в экранные
        screen_x1 = point1[0] + WIDTH // 2
        screen_y1 = HEIGHT // 2 - point1[1]
        screen_x2 = point2[0] + WIDTH // 2
        screen_y2 = HEIGHT // 2 - point2[1]
        
        # TODO: Нарисуйте линию между точками
        pygame.draw.line(
            surface=screen,
            color=(150, 150, 255),  # Светло-синий
            start_pos=(screen_x1, screen_y1),
            end_pos=(screen_x2, screen_y2),
            width=2
        )

def spawn_coin():
    """Создаёт новую точку в случайном месте"""
    # TODO: Случайные координаты в пределах границ
    x = random.randint(-MAX_X, MAX_X)  # От -MAX_X до MAX_X
    y = random.randint(-MAX_Y, MAX_Y)  # От -MAX_Y до MAX_Y
    
    # TODO: Случайный цвет
    color = random.choice(COIN_COLORS)
    
    # TODO: Создайте словарь с данными точки
    coin = {
        'x': x,
        'y': y,
        'color': color,
        'radius': COIN_RADIUS
    }
    
    return coin

def draw_coins(screen, coins):
    """Рисует все точки на экране"""
    for coin in coins:
        # TODO: Преобразуйте математические координаты в экранные
        screen_x = coin['x'] + WIDTH // 2
        screen_y = HEIGHT // 2 - coin['y']
        
        # TODO: Нарисуйте точку
        pygame.draw.circle(
            surface=screen,
            color=coin['color'],
            center=(screen_x, screen_y),
            radius=coin['radius']
        )

def draw_info(screen, x, y, direction, linear_speed, angular_speed, mode, multiplier, score, time_left):
    lines = [
        f"Счёт: {score}",
        f"Время: {int(time_left)}с",
        f"Позиция: ({int(x)}, {int(y)})",
        f"Направление: {int(direction % 360)}°",
        f"Линейная скорость: {int(linear_speed)} пикс/с",
        f"Угловая скорость: {int(angular_speed)} град/с",
        f"Режим: {mode}",
        # TODO: Добавьте строку с множителем скорости
        f"Скорость: {int(multiplier * 100)}%",
        "Tab - режим, +/- скорость, R - сброс"
    ]
    
    y_offset = 10
    for line in lines:
        text_surface = font.render(line, True, BLACK)
        screen.blit(text_surface, (10, y_offset))
        y_offset += 25
        pass

def restart_game():
    """Перезапускает игру"""
    global score, coins, game_start_time, robot_x, robot_y, robot_direction, trail
    
    score = 0
    
    coins.clear()
    
    for i in range(5):
        coins.append(spawn_coin())
    
    robot_x = 0
    robot_y = 0
    robot_direction = 0
    
    trail.clear()
    
    game_start_time = pygame.time.get_ticks()
    
    print("Игра перезапущена!")

trail = []
MAX_TRAIL_LENGTH = 100

coins = []

score = 0

COIN_COLORS = [
    (255, 200, 0),   # Золотой
    (0, 200, 255),   # Голубой
    (255, 100, 200), # Розовый
    (150, 255, 100)  # Зелёный
]

COIN_RADIUS = 8 
ROBOT_RADIUS = 15  

for i in range(5): 
    coins.append(spawn_coin())

game_start_time = pygame.time.get_ticks()  # pygame.time.get_ticks()

GAME_DURATION = 60  # Например, 60 секунд

running = True
while running:
    dt = clock.tick(60) / 1000.0
    elapsed_time = (pygame.time.get_ticks() - game_start_time) / 1000  # Секунды

    remaining_time = GAME_DURATION - elapsed_time

    if remaining_time <= 0:
        # Игра окончена!
        print(f"Игра окончена! Финальный счёт: {score}")
        remaining_time = 0
        time.sleep(10)
        restart_game()
        # TODO: Можно остановить игру или перезапустить


    keys = pygame.key.get_pressed()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                mouse_screen_x, mouse_screen_y = event.pos
        
                robot_x = mouse_screen_x - WIDTH // 2
                robot_y = HEIGHT // 2 - mouse_screen_y
            if event.button == 2:  # Средняя кнопка
                robot_x = 0
                robot_y = 0
                print("Возврат в центр")
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_TAB:
                if control_mode == "keyboard":
                    control_mode = "mouse"
                    print("Режим: следование за мышью")
                else:
                    control_mode = "keyboard"
                    print("Режим: управление клавиатурой")

            if event.key == pygame.K_r:
                restart_game()
            
            if event.key == pygame.K_EQUALS:
                speed_multiplier += 0.25
                speed_multiplier = min(3.0, speed_multiplier)
                print(f"Скорость: {speed_multiplier * 100:.0f}%")

            elif event.key == pygame.K_MINUS:
                speed_multiplier -= 0.25
                speed_multiplier = max(0.25, speed_multiplier)
                print(f"Скорость: {speed_multiplier * 100:.0f}%")
   
    linear_speed = 0
    angular_speed = 0

    # TODO: Проверяем текущий режим и выполняем соответствующий код
    if control_mode == "keyboard":
        keys = pygame.key.get_pressed()
    
        if keys[pygame.K_w] and keys[pygame.K_s]:
            linear_speed = 0
        elif keys[pygame.K_w]:
            # TODO: Умножьте базовую скорость на множитель
            linear_speed = 150 * speed_multiplier
        elif keys[pygame.K_s]:
            linear_speed = -100 * speed_multiplier
    
        if keys[pygame.K_a]:
            angular_speed = 120 * speed_multiplier
        if keys[pygame.K_d]:
            angular_speed = -120 * speed_multiplier

    elif control_mode == "mouse":
        # TODO: Код следования за мышью (из задания 2.2)
        # Получаем позицию курсора
        mouse_screen_x, mouse_screen_y = pygame.mouse.get_pos()
        target_x = mouse_screen_x - WIDTH // 2
        target_y = HEIGHT // 2 - mouse_screen_y
    
        # Вычисляем расстояние и угол
        dx = target_x - robot_x
        dy = target_y - robot_y
        distance = math.hypot(dx, dy)
    
        # Вычисляем угол к цели
        angle_to_target_rad = math.atan2(dy, dx)
        angle_to_target_deg = math.degrees(angle_to_target_rad)
    
        # Вычисляем разницу углов
        angle_difference = angle_to_target_deg - robot_direction
    
        # Нормализуем разницу к диапазону -180 до 180
        if angle_difference > 180:
            angle_difference -= 360
        elif angle_difference < -180:
            angle_difference += 360
    
        # Устанавливаем угловую скорость для плавного поворота
        if abs(angle_difference) > 5:
            angular_speed = (angle_difference * 3) * speed_multiplier
        else:
            angular_speed = 0
    
        # Двигаемся к цели только если смотрим туда
        if distance > 5 and abs(angle_difference) < 30:
            linear_speed = 150 * speed_multiplier
        else:
            linear_speed = 0


    coins_to_remove = []  

    for coin in coins:
        dx = coin['x'] - robot_x
        dy = coin['y'] - robot_y
        distance = math.hypot(dx, dy)

        if distance < ROBOT_RADIUS + coin['radius']:
            # Собрали точку!
            coins_to_remove.append(coin)
            score += 1

    for coin in coins_to_remove:
        coins.remove(coin)

    if len(coins) < 5:
        coins.append(spawn_coin())



    trail.append((robot_x, robot_y))
    if len(trail) > MAX_TRAIL_LENGTH:
        trail.pop(0)



# Ограничение координат
    if robot_x > MAX_X:
        robot_x = MAX_X
    elif robot_x < -MAX_X:
        robot_x = -MAX_X

    if robot_y > MAX_Y:
        robot_y = MAX_Y
    elif robot_y < -MAX_Y:
        robot_y = -MAX_Y

    robot_direction += angular_speed*dt
    angle_rad = math.radians(robot_direction)

    robot_x += math.cos(angle_rad)*dt*linear_speed
    robot_y += math.sin(angle_rad)*dt*linear_speed
    screen.fill(WHITE)
    draw_trail(screen, trail)
    print(f"Телепортация в ({int(robot_x)}, {int(robot_y)})")
    draw_robot(screen=screen, x=robot_x, y=robot_y, direction=robot_direction, radius=15)
    draw_coins(screen, coins)
    draw_info(screen, robot_x, robot_y, robot_direction, linear_speed, angular_speed, control_mode, speed_multiplier, score, time_left=elapsed_time)

    pygame.display.flip()
    
    

pygame.quit()

Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортация в (0, 0)
Телепортац